In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [12]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [13]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [14]:
# reflect an existing database into a new model

Base=automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [15]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [16]:
# Save references to each table

Measurement=Base.classes.measurement
Station=Base.classes.station

In [17]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [18]:
inspector=inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [19]:
columns = inspector.get_columns('measurement')

for x in columns:
    print(x['name'], x["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Climate Analysis

In [22]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
engine.execute("select * from Measurement limit 12").fetchall()
session.query(func.count(Measurement.date)).all()


[(19550)]

In [23]:
# Calculate the date 1 year ago from the last data point in the database
date_calc = session.query(func.max(Measurement.date)).all()
date_calc


[('2017-08-23')]

In [24]:
# Perform a query to retrieve the data and precipitation scores
calc = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= "2018-05-18").\
filter(Measurement.date<="2019-05-18").\
group_by(Measurement.date).all()

calc


[]

In [25]:
# Save the query results as a Pandas DataFrame and set the index to the date column
calc_df = pd.DataFrame(calc, columns = ['date', 'prcp'])


In [26]:
# Sort the dataframe by date
calc_df.sort_values(by='date', ascending=True)
calc_df.rename(columns={0:"Date", 1: "Precipitation"},inplace=True)
calc_df.set_index('date')
calc_df

,date,prcp


In [38]:
# Use Pandas Plotting with Matplotlib to plot the data

calc_df.plot()
plt.figure(1)

TypeError: no numeric data to plot

In [28]:
# Use Pandas to calcualte the summary statistics for the precipitation data

calc_df.describe()

,date,prcp
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [29]:
# Design a query to show how many stations are available in this dataset?
stations = session.query(func.count(Station.station)).all()

stations

[(9)]

In [30]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
stations_active = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
stations_active

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [31]:
stations_active_most = stations_active[0]
stations_active_most

('USC00519281', 2772)

In [33]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
calc_2 = session.query(Measurement.station, func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).filter(Measurement.station == stations_active_most[0]).all()
calc_2

[('USC00519281', 54.0, 85.0, 71.66378066378067)]

In [34]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

stations_active_most = stations_active[0] 
stations_active_most

('USC00519281', 2772)

In [35]:
stations_active_temperature = session.query(Measurement.tobs).\
                        filter(Measurement.date >= "2018-05-18").\
                       filter(Measurement.station == "USC00519281").all()
stations_active_temperature

[]

In [37]:
len(stations_active_temperature)

temperature =[x[0] for x in stations_active_temperature]
temperature

[]

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
